In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 6
fig_height = 4
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2024/Julia"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
abstract type Name end

typeof(Name)

DataType

In [3]:
abstract type SubtypeName <: Name end

In [4]:
supertype(Name)

subtypes(Name)

fieldnames(Int)

methodswith(Name)

@show Name

Name = Name


Name

In [5]:
Int >: Int

true

In [6]:
struct MyType
	x::Int
	y::Int
end

fieldnames(MyType)

(:x, :y)

In [7]:
z = MyType(1, 2)
println(z.x)

1


In [8]:
mutable struct MutableType
    x::Int
    y::Int
end

m = MutableType(3, 4)
m.x = 10  # フィールドxの値を変更可能

10

In [9]:
MyType(x) = MyType(x, 0)

MyType

In [10]:
function process(x::Union{Int, String})
    if x isa Int
        println("The integer is $x")
    elseif x isa String
        println("The string is $x")
    end
end

process(3)

The integer is 3


In [11]:
struct Point{T, U}
    x::T
    y::U
end

In [12]:
p1 = Point{Int, Float64}(3, 4.5)  # 明示的に型を指定
p2 = Point(3, 4.5)  # 型推論により自動的に型が決まる
p2

Point{Int64, Float64}(3, 4.5)

In [13]:
function distance(p::Point{T, U}) where {T, U}
    sqrt(p.x^2 + p.y^2)
end

distance(p2)

5.408326913195984

In [14]:
struct Rectangle{T}
    width::T
    height::T
end

# 型推論によるインスタンス生成
r1 = Rectangle(3.0, 4.0)

# 型指定によるインスタンス生成
r2 = Rectangle{Int}(3, 4)

Rectangle{Int64}(3, 4)

In [15]:
# 多重ディスパッチの実践

function area(r::Rectangle{T}) where T
    r.width * r.height
end

println(area(r1))  # 出力: 12.0
println(area(r2))  # 出力: 12

12.0
12


In [16]:
Point{Int} <: Point

true

In [17]:
typeof(Point)

UnionAll

In [18]:
Point{Int} <: Point{Float64}

false

In [19]:
function distance(p::Point{<:Number})
    sqrt(p.x^2 + p.y^2)
end

function distance(p::Point{T}) where T <: Number
    sqrt(p.x^2 + p.y^2)
end

distance (generic function with 2 methods)

In [20]:
p1 = Point(3, 4)  # Point{Int}
distance(p1)  # 呼び出し成功

5.0

In [21]:
isa(Float64, Type{Float64})  # Float64 は Type{Float64} 型のインスタンス
isa(Real, Type{Float64})

false